In [2]:
import nltk


In [3]:
#module that does the tokenization,stemming and bag_of_words
import numpy as np


from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenize(sentence):#breaks down all the words into tokens
    return nltk.word_tokenize(sentence)

def stem(word):#stems down each word i.e sleeping => sleep
    return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, all_words):
    tokenized_sentence = [stem(w) for w in tokenized_sentence]

    bag = np.zeros(len(all_words), dtype=np.float32)
    for idx, w in enumerate(all_words):
        if w in tokenized_sentence:
            bag[idx] = 1.0
    return bag

In [6]:
#module that takes in the training data, gets the tags(labels), and all the words
import json

with open('intents.json', 'r') as f:
    intents = json.load(f)
#print(intents)

all_words = []
tags = []
xy = []

for intent in intents['intents']:
    tag = intent["tag"]
    tags.append(tag)
    
    #patterns asper things that could be prompted
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w,tag)) #basically words asscociated with a particular tag

ignore_words = ['?', '!', ',', '.']
all_words = [stem(w) for w in all_words if w not in ignore_words]

all_words = sorted(set(all_words)) #removes duplicates
tags = sorted(set(tags))
print(xy)

[(['What', 'are', 'the', 'core', 'beliefs', 'of', 'Catholicism', '?'], 'catholic_beliefs'), (['Tell', 'me', 'about', 'Catholic', 'beliefs'], 'catholic_beliefs'), (['What', 'does', 'the', 'Catholic', 'Church', 'teach', '?'], 'catholic_beliefs'), (['How', 'many', 'sacraments', 'are', 'there', 'in', 'the', 'Catholic', 'Church', '?'], 'sacraments'), (['Can', 'you', 'list', 'the', 'Catholic', 'sacraments', '?'], 'sacraments'), (['What', 'is', 'the', 'significance', 'of', 'sacraments', '?'], 'sacraments'), (['Who', 'is', 'the', 'current', 'Pope', '?'], 'pope'), (['Tell', 'me', 'about', 'the', 'role', 'of', 'the', 'Pope', 'in', 'Catholicism'], 'pope'), (['How', 'is', 'the', 'Pope', 'chosen', '?'], 'pope'), (['What', 'are', 'some', 'common', 'Catholic', 'prayers', '?'], 'prayer'), (['How', 'should', 'Catholics', 'pray', '?'], 'prayer'), (['Tell', 'me', 'about', 'the', 'importance', 'of', 'prayer', 'in', 'Catholicism'], 'prayer'), (['What', 'is', 'the', 'Catholic', 'stance', 'on', 'morality', '

In [10]:
#module that contains the model
import torch
import torch.nn as nn

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet,self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        
        return(out)

In [16]:
#training of the model
from torch.utils.data import Dataset, DataLoader

X_train = [] #words
y_train = [] #labels

for (pattern_sentence, tag) in xy:
    bag  = bag_of_words(pattern_sentence, all_words)
    
    X_train.append(bag)

    label = tags.index(tag)

    y_train.append(label) 

X_train = np.array(X_train)
y_train = np.array(y_train)

class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.n_samples

batch_size = 8
hidden_size = 8
output_size = len(tags)
input_size = len(X_train[0])
learning_rate = 0.001  # Decreased learning rate
num_epochs = 1000

dataset = ChatDataset()
train_loader = DataLoader(dataset = dataset, batch_size=batch_size, shuffle =True, num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuralNet(input_size, hidden_size, output_size).to(device)

#loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    for(words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(torch.int64) 
        labels = labels.to(device)

        outputs = model(words)
        loss = criterion(outputs, labels)

        optimizer.zero_grad() #empties gradient
        loss.backward() #calculates backpropagation
        optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'epoch {epoch+1}/{num_epochs} , loss = {loss.item():.4f}')

print(f'Final loss,  loss = {loss.item():.4f}')

epoch 100/1000 , loss = 0.7352
epoch 200/1000 , loss = 0.3667
epoch 300/1000 , loss = 0.0483
epoch 400/1000 , loss = 0.0144
epoch 500/1000 , loss = 0.0142
epoch 600/1000 , loss = 0.0025
epoch 700/1000 , loss = 0.0038
epoch 800/1000 , loss = 0.0042
epoch 900/1000 , loss = 0.0013
epoch 1000/1000 , loss = 0.0009
Final loss,  loss = 0.0009


In [17]:
#module that saves to file
data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "output_size": output_size,
    "hidden_size" : hidden_size,
    "all_words": all_words,
    "tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'Training complete! File saved to {FILE}')

Training complete! File saved to data.pth


In [18]:
#CHATBOT
import random
# import json
# import torch
# import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('intents.json','r') as f:
    intents = json.load(f)

FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data["all_words"]
tags = data["tags"]
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()


bot_name = "Chima"

"""
print("Let's chat: type 'quit' to exit")
while True:

    sentence = input('You: ')
    if sentence == 'quit':
        break
        """

def get_response(msg):
    sentence = tokenize(msg)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X= torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim = 1)
    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim = 1)
    prob = probs[0][predicted.item()]
   
    if prob.item()> 0.75:
        for intent in intents["intents"]:
            if tag == intent["tag"]:
                return random.choice(intent['responses'])
    return "I do not understand..."

In [24]:
from tkinter import *


BG_GRAY = "#ABB289"
BG_COLOR = "black"
TEXT_COLOR = "#EAECEE"

FONT = "Times"
FONT_BOLD = "Times 13 bold"

class ChatApplication:
    def __init__(self):
        self.window = Tk()
        self._setup_main_window()
    
    def run(self):
        self.window.mainloop()
        
    def _setup_main_window(self):
        self.window.title("CatholiChat!")
        self.window.resizable(width=True, height=True)
        self.window.configure(width=600, height=550, bg=BG_COLOR)

        
        #head label
        head_label = Label(self.window, bg = BG_COLOR, fg = TEXT_COLOR,
                           text = "Welcome",font=FONT_BOLD,pady=10)
        head_label.place(relwidth=1)
        
        #divider
        line = Label(self.window, width=450, bg=BG_GRAY)
        line.place(relwidth=1, rely=0.07, relheight=0.012)
        
        #text
        self.text_widget = Text(self.window, width=20, height=2, bg=BG_COLOR,
                              fg=TEXT_COLOR,font=FONT,padx=5,pady=5) 
        self.text_widget.place(relheight=0.745, relwidth=1,rely=0.08)
        self.text_widget.configure(cursor="arrow",state=DISABLED)
        
        #SCROLL BAR
        scrollbar = Scrollbar(self.text_widget)
        scrollbar.place(relheight=1,relx=0.974)
        scrollbar.configure(command=self.text_widget.yview)
        
        #bottom label
        bottom_label = Label(self.window, bg=BG_GRAY, height=80)
        bottom_label.place(relwidth=1, rely=0.825)
        
        #message entry box
        self.msg_entry = Entry(bottom_label, bg="#2C3E50",fg=TEXT_COLOR,
                              font=FONT)
        self.msg_entry.place(relwidth=0.74, relheight=0.06, rely=0.008,
                            relx=0.011)
        self.msg_entry.focus()
        self.msg_entry.bind("<Return>",self._on_enter_pressed)
        
        send_button = Button(bottom_label, text="Send",font=FONT_BOLD,
                            width=20, bg=BG_GRAY,
                             command=lambda: self._on_enter_pressed(None))
        send_button.place(relx=0.77, rely=0.008, relheight=0.06,relwidth=0.22)
        
    def _on_enter_pressed(self, event):
        msg = self.msg_entry.get()
        self._insert_message(msg, "You")
    
    def _insert_message(self, msg, sender):
        if not msg:
            return
        self.msg_entry.delete(0, END)
        msg1 = f"{sender}: {msg}\n\n"
        self.text_widget.configure(state=NORMAL)
        self.text_widget.insert(END, msg1)
        self.text_widget.configure(state=DISABLED)
        
        msg2 = f"{bot_name}: {get_response(msg)}\n\n"  #part that interacts with the get_response function
        self.text_widget.configure(state=NORMAL)
        self.text_widget.insert(END, msg2)
        self.text_widget.configure(state=DISABLED)
        
        self.text_widget.see(END)
        
if __name__ == "__main__":
    app = ChatApplication()
    app.run()